In [66]:
import pymysql
import json
from functools import partial
import csv
import networkx as nx
import freeman as fm
from random import random, choices
def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)
            
def return_db_results(connection, query, args=None):#Gambiarra para pegar o id
    with connection.cursor() as cursor:
        cursor.execute(query, args)
        for result in cursor:
            return result #retorna logo o primeiro

def multiple_db_results(connection, query, args=None):#Gambiarra para pegar o id
    with connection.cursor() as cursor:
        cursor.execute(query, args)
        res = cursor.fetchall()
        return res

connection = pymysql.connect(
    host='localhost',
    user='root',
    password='vlm1998',
    database='redes')

db = partial(run_db_query, connection)

In [69]:
def add_node(g, n):
    g.add_node(n)
    g.nodes[n]['pos'] = (random(), random())
    g.nodes[n]['label'] = n
    g.nodes[n]['color'] = (255, 0, 0)

def add_edge(g, n, m):
    print("De: ", n, "para: ", m)
    g.add_edge(n, m)

def get_team_name(connection, id_team):
    with connection.cursor() as cursor:
        cursor.execute("SELECT nome FROM times WHERE id_time = %s", (id_team))
        res = cursor.fetchone()
        if res:
            return res
        else:
            return -1
def list_all_teams(connection):
    with connection.cursor() as cursor:
        cursor.execute("SELECT nome FROM times")
        res = cursor.fetchall()
        return res
def list_all_transfers(connection):
    with connection.cursor() as cursor:
        cursor.execute("SELECT * FROM transferencia")
        res = cursor.fetchall()
        return res
def get_from_where(connection, id_jogador, ano):
    with connection.cursor() as cursor:
        cursor.execute("SELECT times.nome FROM waived INNER JOIN times USING(id_time) WHERE id_jogador = %s AND ano = %s ", (id_jogador, str(ano)))
        res = cursor.fetchone()
        if res:
            return res
        else:
            return -1


In [71]:
a = fm.Animation()
g = nx.DiGraph()
a.rec(g)

transfers = list_all_transfers(connection)
teams = list_all_teams(connection)

for team in teams:
    add_node(g, team)

    
#Existem casos em que um time assina com um jogador e na mesma temporada vende ele, vamos ignorar estes casos por enquanto
for transfer in transfers:
    if(transfer[3] == 2019):
        antigo = get_from_where(connection, transfer[1], transfer[3])
        novo = get_team_name(connection, transfer[2])
        if(antigo != -1) and (novo != -1):
            if(antigo != novo):
                add_edge(g, antigo, novo)

a.rec(g)
a.play()

Miami Heat Houston Rockets
De:  ('Miami Heat',) para:  ('Houston Rockets',)
Golden State Warriors Sacramento Kings
De:  ('Golden State Warriors',) para:  ('Sacramento Kings',)
Cleveland Cavaliers Toronto Raptors
De:  ('Cleveland Cavaliers',) para:  ('Toronto Raptors',)
Oklahoma City Thunder LA Clippers
De:  ('Oklahoma City Thunder',) para:  ('LA Clippers',)
Philadelphia 76ers Oklahoma City Thunder
De:  ('Philadelphia 76ers',) para:  ('Oklahoma City Thunder',)
San Antonio Spurs Milwaukee Bucks
De:  ('San Antonio Spurs',) para:  ('Milwaukee Bucks',)
Phoenix Suns Milwaukee Bucks
De:  ('Phoenix Suns',) para:  ('Milwaukee Bucks',)
Chicago Bulls Cleveland Cavaliers
De:  ('Chicago Bulls',) para:  ('Cleveland Cavaliers',)
Sacramento Kings Houston Rockets
De:  ('Sacramento Kings',) para:  ('Houston Rockets',)
Phoenix Suns Chicago Bulls
De:  ('Phoenix Suns',) para:  ('Chicago Bulls',)
Phoenix Suns Los Angeles Lakers
De:  ('Phoenix Suns',) para:  ('Los Angeles Lakers',)
New York Knicks Portland T

In [72]:
connection.close()